In [4]:
OPENQASM 3.0;
// The 'stdgates.inc' include is supported, and the gates are only available
// if it has correctly been included.
include "stdgates.inc";

// Parametrised inputs are supported.
input float[64] a;

qubit[3] q;
bit[2] mid;
bit[3] out;

// Aliasing and re-aliasing are supported.
let aliased = q[0:1];

// Parametrised gates that make use of the stdlib.
gate my_gate(a) c, t {
  gphase(a / 2);
  ry(a) c;
  cx c, t;
}

// Gate modifiers work as well; this gate is equivalent to `p(-a) c;`.
gate my_phase(a) c {
  ctrl @ inv @ gphase(a) c;
}

// We handle mathematical expressions on gate creation and complex indexing
// of temporary collections.
my_gate(a * 2) aliased[0], q[{1, 2}][0];
measure q[0] -> mid[0];
measure q[1] -> mid[1];

while (mid == "00") {
  reset q[0];
  reset q[1];
  my_gate(a) q[0], q[1];
  // We support the builtin mathematical symbols.
  my_phase(a - pi/2) q[1];
  mid[0] = measure q[0];
  mid[1] = measure q[1];
}

// The condition resolver can also handle simple cases that don't look
// _exactly_ like equality conditions.
if (mid[0]) {
  // There is limited support for aliasing within nested scopes.
  let inner_alias = q[{0, 1}];
  reset inner_alias;
}

out = measure q;

SyntaxError: invalid syntax (3231794812.py, line 1)

In [16]:
import qiskit.qasm3

program =  """   
OPENQASM 3;
    include "stdgates.inc";

qubit[3] q;
bit[3] out;

h q[0];
cnot q[0], q[1];
cnot q[1], q[2];

c = measure q;
"""
circuit = qiskit.qasm3.loads(program)
circuit.draw("mpl")

QASM3ImporterError: "9,0: gate 'cnot' is not defined."

In [11]:
from qiskit import qasm3, QuantumCircuit, QuantumRegister, ClassicalRegister

# Build the circuit
qreg = QuantumRegister(3)
creg = ClassicalRegister(3)
qc = QuantumCircuit(qreg, creg)
with qc.switch(creg) as case:
    with case(0):
        qc.x(0)
    with case(1, 2):
        qc.x(1)
    with case(case.DEFAULT):
        qc.x(2)

# Export to an OpenQASM 3 string.
qasm_string = qasm3.dumps(qc, experimental=qasm3.ExperimentalFeatures.SWITCH_CASE_V1)


$H^{\prime}(1)=U^{\dagger} (1)H_{1}U(1) $
